In [1]:
from IPython.display import display, HTML
display(HTML('<style>.container { width:90% !important; }</style>'))  # makes it wider

import requests
import schedule
import time
import traceback
import yaml
import sys
import platform

from configs import directory
from logger import Logger
from utils import *
from create_api import CreateApi 
from os.path import join, dirname


***** RUNNING ON : local *****


In [128]:
def api_executor(api_info):
    import os
    _file_path = join(os.path.abspath(""), 'ml_executor.py')  # join(dirname(__file__), api_info['py_file'])
    _py = callfunc(_file_path)
    print(_py)
    _func = [o[1] for o in getmembers(_py) if o[0] == api_info['function']][0]
    print(_file_path, _py,  _func)
    api = CreateApi(host=api_info['host'],
                    port=api_info['port'],
                    function=_func,
                    api_name=api_info['api_name'],
                    parameters=list(getargspec(_func)[0]))
    api.init_api()

In [129]:
def get_api_url(host, port, api_name):
    host = '0.0.0.0' if host is None else host
    return "http://" + host + ":" + str(port) + "/" + api_name

In [134]:
class CreateJobs:
    def __init__(self, jobs, job_name):
        self.job_name = job_name
        self.jobs_yaml = jobs
        self.job = self.jobs_yaml[job_name]
        self.api_info = list(filter(lambda x: x['api_name'] == self.job['api'], read_yaml(directory + 'apis.yaml')))[0]
        self.url = get_api_url(host=self.api_info['host'],
                               port=self.api_info['port'],
                               api_name=self.api_info['api_name'])
        self.time = str(self.job['time'])
        self.schedule = None

    def jobs(self):
        for j in self.job['execute']:
            try:
                requests.get(url=self.url, params=j['params'])
            except Exception as e:
                print(e)
                api_executor(self.api_info)
                requests.get(url=self.url, params=j['params'])

    def stop_job(self):
        if bool(self.jobs_yaml[self.job_name]['active']):  # if there is active job update ml_execute.yaml
            self.jobs_yaml[self.job_name]['active'] = False
            with open(directory + "ml_execute.yaml", 'w') as file:
                yaml.dump(self.jobs_yaml, file)
            time.sleep(10)  # make sure it is stop

    def job_schedule(self):
        self.stop_job()
        print("job is initialized!!!")
        print(platform.node())
        print(self.time)
        if self.job['day'] != 'min':
            {'monday': schedule.every().monday,
             'tuesday': schedule.every().tuesday,
             'wednesday': schedule.every().wednesday,
             'thursday': schedule.every().thursday,
             'friday': schedule.every().friday,
             'saturday': schedule.every().saturday,
             'sunday': schedule.every().sunday,
             'day': schedule.every().day,
             'month': schedule.every(30).days
             }[self.job['day']].at(self.time).do(self.jobs)
        else:
            schedule.every(2).minutes.do(self.jobs)

In [135]:
j = CreateJobs(read_yaml(directory + 'ml_execute.yaml'), 'test')

In [136]:
j.api_info

{'port': 3537,
 'host': 'Mac-MacBook-Air.local',
 'api_name': 'test',
 'py_file': 'ml_executor',
 'function': 'test_function',
 'params': ['param1', 'param2']}

In [137]:
def ml_execute(job, process):
    Logger(job + 'schedule_')
    print("Creating Jobs")
    j = CreateJobs(read_yaml(directory + 'ml_execute.yaml'), job)
    if process == 'start':
        j.job_schedule()
        while True:
            schedule.run_pending()
            jobs = read_yaml(directory + 'ml_execute.yaml')
            print("job is working")
            if not bool(jobs[job]):
                print("job is stoped !!")
                break
            time.sleep(5)
    if process == 'end':
        j.stop_job()

In [138]:
ml_execute('test', 'start')

INFO:schedule:Running job Every 2 minutes do jobs() (last run: [never], next run: 2020-05-08 00:40:31)


Creating Jobs
job is initialized!!!
Mac-MacBook-Air.local
02:01


INFO:schedule:Running job Every 5 seconds do job() (last run: 2020-05-08 00:40:29, next run: 2020-05-08 00:40:34)
INFO:schedule:Running job Every 5 seconds do job() (last run: 2020-05-08 00:40:29, next run: 2020-05-08 00:40:34)
INFO:schedule:Running job Every 2 minutes do jobs() (last run: [never], next run: 2020-05-08 00:40:44)
INFO:schedule:Running job Every 2 minutes do jobs() (last run: [never], next run: 2020-05-08 00:45:36)
INFO:schedule:Running job Every 2 minutes do jobs() (last run: [never], next run: 2020-05-08 00:51:19)
INFO:schedule:Running job Every 2 minutes do jobs() (last run: [never], next run: 2020-05-08 00:53:25)


I'm working...
I'm working...


INFO:schedule:Running job Every 2 minutes do jobs() (last run: [never], next run: 2020-05-08 00:53:48)
INFO:schedule:Running job Every 2 minutes do jobs() (last run: [never], next run: 2020-05-08 00:54:09)
INFO:schedule:Running job Every 2 minutes do jobs() (last run: [never], next run: 2020-05-08 00:54:33)
INFO:schedule:Running job Every 2 minutes do jobs() (last run: [never], next run: 2020-05-08 02:03:00)
INFO:schedule:Running job Every 2 minutes do jobs() (last run: [never], next run: 2020-05-08 02:05:20)
INFO:schedule:Running job Every 2 minutes do jobs() (last run: [never], next run: 2020-05-08 02:06:41)


...

INFO:schedule:Running job Every 2 minutes do jobs() (last run: [never], next run: 2020-05-08 02:11:26)
INFO:schedule:Running job Every 2 minutes do jobs() (last run: 2020-05-08 02:09:27, next run: 2020-05-08 02:11:27)
INFO:schedule:Running job Every 2 minutes do jobs() (last run: 2020-05-08 02:09:27, next run: 2020-05-08 02:11:27)
INFO:schedule:Running job Every 2 minutes do jobs() (last run: 2020-05-08 02:09:27, next run: 2020-05-08 02:11:27)
INFO:schedule:Running job Every 2 minutes do jobs() (last run: 2020-05-08 02:09:27, next run: 2020-05-08 02:11:27)
INFO:schedule:Running job Every 2 minutes do jobs() (last run: 2020-05-08 02:09:27, next run: 2020-05-08 02:11:27)
INFO:schedule:Running job Every 2 minutes do jobs() (last run: 2020-05-08 02:09:27, next run: 2020-05-08 02:11:27)
INFO:schedule:Running job Every 2 minutes do jobs() (last run: 2020-05-08 02:09:27, next run: 2020-05-08 02:11:27)
INFO:schedule:Running job Every 2 minutes do jobs() (last run: 2020-05-08 02:09:27, next run

I'm working...
I'm working...
job is working


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

INFO:root:
Unfortunately, your original traceback can not be constructed.



Traceback (most recent call last):
  File "/Users/mac/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-138-0bd6b74b388d>", line 1, in <module>
    ml_execute('test', 'start')
  File "<ipython-input-137-e134e8ea4c1b>", line 14, in ml_execute
    time.sleep(5)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/mac/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2040, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/mac/opt/anaconda3/lib/python3.7/site-packages/IPython/core/ultratb.py", line 1101, in get_records
    return _fixed_getinnerfram

KeyboardInterrupt: 

In [125]:
_file_path = join(os.path.abspath(""), 'ml_executor.py')  # join(dirname(__file__), api_info['py_file'])
_py = callfunc(_file_path)
_py

<module 'ml_executor' from '/Users/mac/Documents/GitHub/anomaly_detection_framework/ml_executor.py'>

In [116]:
_file_path = join(os.path.abspath(""), 'ml_executor.py')
print("_file_path ::::::", _file_path)
_py = callfunc(_file_path)

_file_path :::::: /Users/mac/Documents/GitHub/anomaly_detection_framework/ml_executor.py


In [117]:
from inspect import getmembers, getargspec
sorted([o[0] for o in getmembers(_py)])

['CreateJobs',
 'Logger',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'directory',
 'get_api_url',
 'ml_execute',
 'platform',
 'read_yaml',
 'requests',
 'schedule',
 'sys',
 'test_function',
 'time',
 'traceback',
 'yaml']

In [142]:
str(datetime.datetime.now())[0:16]

'2020-05-12 11:58'

In [147]:
test = {'a': 1, 'b': None}

In [155]:
if not test.get('c', None):
    print("yess")

yess


In [174]:
today = datetime.datetime.now()
today + datetime.timedelta(days=-today.weekday())

datetime.datetime(2020, 5, 11, 13, 46, 4, 757831)

In [172]:
datetime.timedelta(days=-today.weekday())

datetime.timedelta(days=-1)

In [175]:
today + datetime.timedelta(days=0)

datetime.datetime(2020, 5, 12, 13, 46, 4, 757831)

In [178]:
datetime.datetime.strptime(str(datetime.datetime.now())[0:16],
                                                 "%Y-%m-%d %H:%M") + datetime.timedelta(minutes=2)

datetime.datetime(2020, 5, 12, 15, 43)

In [200]:
log_main_path = 'log'
import yaml
from numpy import arange
from configs import log_main_path
from utils import read_yaml

In [246]:
class LoggerProcess:
    def __init__(self, job=None, model=None, total_process=None):
        self.log = log_main_path + "process.yaml"
        self.job = job
        self.model = model
        self.total_process = total_process
        self.ratios = [int(i * total_process) for i in arange(0.1, 1.1, 0.1)] if total_process else None
        self.count = 0

    def write(self):
        file = read_yaml(self.log)
        file[self.job][self.model] = self.count / self.total_process
        with open(self.log, 'w') as f:
            yaml.dump(file, f)

    def counter(self):
        self.count += 1
        if self.count in self.ratios:
            self.write()
            
    def regenerate_file(self):
        file = read_yaml(self.log)
        for j in file:
            if j == self.job:
                for m in file[j]:
                    file[j][m] = 0
        with open(self.log, 'w') as f:
            yaml.dump(file, f)
        

In [247]:
log = LoggerProcess('train', 'lstm', 1000)

In [248]:
for i in range(1001):
    log.counter()

In [249]:
log.regenerate_file()

In [252]:
def test(*args):
    print(args[0])

In [253]:
test([1,2,3])

[1, 2, 3]


In [254]:
def is_port_in_use(port):
    import socket
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        return s.connect_ex(('localhost', port)) == 0

In [255]:
is_port_in_use(5000)

False

In [257]:
np.random.randint(low=1, high=10, size=1)[0]

6

In [260]:
import os
os.system('docker-compose up')

512

In [3]:
jobs = read_yaml('ml_execute.yaml')

In [12]:
import pandas as pd

In [39]:
params = jobs['parameter_tuning']['execute'][1]['params']
params

{'data_query_path': 'ml3yil_20200429.csv',
 'data_source': 'csv',
 'date': '2020-05-12 11:59',
 'feature': 'KisiSayimi',
 'groups': 'BayiKodu',
 'job': 'parameter_tuning',
 'model': 'prophet',
 'time_indicator': 'Tarih',
 'time_period': None,
 'time_periods': None}

In [79]:
requests.get(url="http://mac-macbook-air.local:6001/ml", params={'job': 'stop', 'model': 'train'})

<Response [200]>

In [34]:
max_date = datetime.datetime.strptime('2020-05-16 21:31:53', "%Y-%m-%d %H:%M:%S")
min_date = datetime.datetime.strptime('2020-05-15 21:31:53', "%Y-%m-%d %H:%M:%S")
print()
range_dates = (max_date - min_date).total_seconds()
range_dates

86400.0

In [51]:
import os
from multiprocessing import Process
import subprocess

In [58]:
heavy_process = Process(
                        target=run_ml_executor,
                        daemon=True
                        )
heavy_process.start()

In [59]:
def run_ml_executor():
    subprocess.run(["docker-compose", "up"])

In [45]:
import subprocess
subprocess.run(["ls", "-l"])

CompletedProcess(args=['ls', '-l'], returncode=0)

In [65]:
def main(**args):
    print(args)

In [67]:
def get_running_pids(process_name, argument=None):
    pids = []
    p = subprocess.Popen(['ps', '-A'], stdout=subprocess.PIPE)
    out, err = p.communicate()
    for line in out.splitlines():
        if process_name in line.decode('utf-8'):
            pid = int(line.decode('utf-8').split(None, 1)[0])
            if argument:
                if len(line.decode('utf-8').split(process_name)) != 0:
                    args = line.decode('utf-8').split(process_name)[1].split(None, 1)
                    if str(argument) in args:
                        print("running job :", line.decode('utf-8'))
                        pids.append(pid)
            else:
                print("running job :", line.decode('utf-8'))
                pids.append(pid)
        else:
            print("no initialized ", argument, " is detected")
    return pids


def kill_process_with_name(process_name, argument=None):
    pids = get_running_pids(process_name, argument=argument)
    if len(pids) != 0:
        for pid in pids:
            os.kill(pid, signal.SIGKILL)
    else:
        print("no running jobs")

In [78]:
kill_process_with_name("test.py", argument=None)

no initialized  None  is detected
no initialized  None  is detected
no initialized  None  is detected
no initialized  None  is detected
no initialized  None  is detected
no initialized  None  is detected
no initialized  None  is detected
no initialized  None  is detected
no initialized  None  is detected
no initialized  None  is detected
no initialized  None  is detected
no initialized  None  is detected
no initialized  None  is detected
no initialized  None  is detected
no initialized  None  is detected
no initialized  None  is detected
no initialized  None  is detected
no initialized  None  is detected
no initialized  None  is detected
no initialized  None  is detected
no initialized  None  is detected
no initialized  None  is detected
no initialized  None  is detected
no initialized  None  is detected
no initialized  None  is detected
no initialized  None  is detected
no initialized  None  is detected
no initialized  None  is detected
no initialized  None  is detected
no initialized

In [76]:
import subprocess
p = subprocess.Popen(['ps', '-A'], stdout=subprocess.PIPE)
out, err = p.communicate()
out.splitlines()

[b'  PID TTY           TIME CMD',
 b'    1 ??       109:01.70 /sbin/launchd',
 b'   94 ??         1:42.87 /usr/sbin/syslogd',
 b'   95 ??        17:19.44 /usr/libexec/UserEventAgent (System)',
 b'   98 ??         1:24.41 /System/Library/PrivateFrameworks/Uninstall.framework/Resources/uninstalld',
 b'   99 ??         0:57.30 /usr/libexec/kextd',
 b'  100 ??       111:22.62 /System/Library/Frameworks/CoreServices.framework/Versions/A/Frameworks/FSEvents.framework/Versions/A/Support/fseventsd',
 b'  101 ??         3:03.83 /System/Library/PrivateFrameworks/MediaRemote.framework/Support/mediaremoted',
 b'  104 ??       168:32.91 /usr/sbin/systemstats --daemon',
 b'  105 ??         4:13.85 /usr/libexec/configd',
 b'  107 ??        21:15.19 /System/Library/CoreServices/powerd.bundle/powerd',
 b'  111 ??        28:44.97 /usr/libexec/logd',
 b'  113 ??         0:35.19 /Library/PostgreSQL/11/bin/postmaster -D/Library/PostgreSQL/11/data',
 b'  116 ??         1:31.10 /usr/libexec/watchdogd',
 b'  

In [80]:
def test():
    print("ok!!!")

In [81]:
import schedule

In [87]:
schedule.every(2).weeks.do(test)

Every 2 weeks do test() (last run: [never], next run: 2020-06-01 22:16:19)

In [88]:
while True:
    schedule.run_pending()
    print("waiting!!")
    time.sleep(10)

waiting!!
waiting!!
waiting!!


KeyboardInterrupt: 